In [ ]:
from pathlib import Path
import datetime

import numpy as np
import yfinance as yf
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

import stock

In [ ]:
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 0.5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

In [ ]:
codes = stock.get_code_list()
all_codes = stock.get_code_list(include_etf=True)

In [ ]:
#while date > datetime.
output_dir = stock.PROJECT_ROOT / "data" / "minutes"

for code in all_codes:
    print("Start downloading : {}".format(code))
    date = datetime.date(2024, 7, 26)
    while date <= datetime.date.today():
        output_path = output_dir / date.strftime("%Y%m%d") / f"{code}_{date.strftime('%Y%m%d')}.arrow"
        if output_path.exists() or date.weekday() >= 5:
            date += datetime.timedelta(days=1)
            continue
        ticker = yf.Ticker(f"{code}.T", session=session)
        df = stock.util.pd_to_pl(ticker.history(
            interval="1m",
            start=date.strftime("%Y-%m-%d"),
            end=(date + datetime.timedelta(days=1)).strftime("%Y-%m-%d"),
        ))
        if len(df) > 0:
            output_path.parent.mkdir(parents=True, exist_ok=True)
            df.write_ipc(output_path)
        date += datetime.timedelta(days=1)